In [331]:
from datetime import datetime
from typing import List, Set
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy import Integer
from sqlalchemy import Boolean
from sqlalchemy import Float
from sqlalchemy import func, text
from sqlalchemy import UniqueConstraint
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship


from sqlalchemy import create_engine

class Base(DeclarativeBase):
    pass

class Company(Base):
    __tablename__ = 'company'
    id: Mapped[int] = mapped_column(primary_key=True)
    #coa_key: Mapped[int] = mapped_column(ForeignKey('chart_of_accounts.id'))
    company_name: Mapped[str] = mapped_column(String(250))
    company_type: Mapped[str] = mapped_column(String(50))
    date: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(1000))

    coa: Mapped['ChartOfAccounts'] = relationship(back_populates='company', uselist=False)  # , primaryjoin=coa_key==ChartOfAccounts.id)

    #__table_args__ = (UniqueConstraint('coa_key'),)
    
    def __repr__(self) -> str:
        return f"Company(id={self.id!r}, company_name={self.company_name!r}, description={self.description!r})"
    

class ChartOfAccounts(Base):
    __tablename__ = 'chart_of_accounts'
    id: Mapped[int] = mapped_column(primary_key=True)
    company_key: Mapped[int] = mapped_column(ForeignKey('company.id'))
    name: Mapped[str] = mapped_column(String(50))

    company: Mapped['Company'] = relationship(back_populates='coa', single_parent=True, foreign_keys=[company_key], uselist=False)
    accounts: Mapped[List['Accounts']] = relationship(back_populates='coa')

    def __repr__(self) -> str:
        return f"ChartOfAccounts(id={self.id!r}, description={self.company!r})" 

def get_parent_coa_key(context):
    parent_account_key = context.compiled_parameters[0]['parent_account_key']
    
    sql = text(f'SELECT coa_key FROM accounts WHERE id={parent_account_key}')
    result = context.connection.execute(sql).fetchone()[0]
    print(result)
    return result

class Accounts(Base):
    __tablename__ = 'accounts'
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    coa_key: Mapped[int] = mapped_column(ForeignKey('chart_of_accounts.id'), default=get_parent_coa_key)
    parent_account_key: Mapped[int] = mapped_column(ForeignKey('accounts.id'), nullable=True)
    name: Mapped[str] = mapped_column(String(50))
    number: Mapped[int] = mapped_column(Integer)
    account_type: Mapped[str] = mapped_column(String(50))
    is_container: Mapped[bool] = mapped_column(Boolean)
    is_root: Mapped[bool] = mapped_column(Boolean)
    date_created: Mapped[datetime] = mapped_column(insert_default=func.now())
    date_modified: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(250))  

    coa: Mapped['ChartOfAccounts'] = relationship(back_populates='accounts')
    children: Mapped[Set['Accounts']] = relationship('Accounts')
    splits: Mapped[Set['Splits']] = relationship('Splits', back_populates='account')

    def __repr__(self) -> str:
        return f"Accounts(id={self.id!r}, description={self.description!r}, container={self.is_container!r}, root={self.is_root!r})"


class JournalEntries(Base):
    __tablename__ = 'journal_entries'
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    date: Mapped[datetime] = mapped_column(insert_default=func.now())
    date_entered: Mapped[datetime] = mapped_column(insert_default=func.now())
    date_modified: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(250))  
    memo: Mapped[str] = mapped_column(String(250))  
    debit: Mapped[float] = mapped_column(Float, nullable=True)
    credit: Mapped[float] = mapped_column(Float, nullable=True)

    splits: Mapped[Set['Splits']] = relationship(back_populates='journal_entry')

    def __repr__(self) -> str:
        return f"JournalEntries(id={self.id!r}, description={self.description!r}, debit={self.debit!r}, credit={self.credit!r})"

class Splits(Base):
    __tablename__ = 'splits'
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    journal_entry_key: Mapped[int] = mapped_column(ForeignKey('journal_entries.id'))
    account_key: Mapped[int] = mapped_column(ForeignKey('accounts.id'))
    date: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(250)) 
    memo: Mapped[str] = mapped_column(String(250)) 
    debit: Mapped[float] = mapped_column(Float)
    credit: Mapped[float] = mapped_column(Float)

    journal_entry: Mapped['JournalEntries'] = relationship(back_populates='splits')
    account: Mapped['Accounts'] = relationship(back_populates='splits')

    def __repr__(self) -> str:
         return f"Splits(id={self.id!r}, description={self.description!r}, debit={self.debit!r}, credit={self.credit!r})"


from sqlalchemy import event
@event.listens_for(Splits.debit, 'set', retval=True)
def Splits_debit_modified_listener(target, value, oldvalue, initiator):
    print('setting to 4')
    return 4

In [332]:
engine = create_engine('sqlite://', echo=True)


In [333]:
Base.metadata.create_all(engine)

2025-05-21 14:53:54,633 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 14:53:54,634 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("company")
2025-05-21 14:53:54,634 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 14:53:54,635 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("company")
2025-05-21 14:53:54,636 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 14:53:54,638 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("chart_of_accounts")
2025-05-21 14:53:54,639 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 14:53:54,640 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("chart_of_accounts")
2025-05-21 14:53:54,641 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 14:53:54,643 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("accounts")
2025-05-21 14:53:54,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 14:53:54,645 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("accounts")
2025-05-21 14:53:54,646 INFO sqlalchemy.engi

In [334]:
from sqlalchemy.orm import Session

with Session(engine) as s:
    
    company = Company(
        company_name='Test Company',
        company_type='LLC',
        date=datetime.now(),
        description='A test company',
        coa = ChartOfAccounts(
            name='Test COA2',
            accounts=[
                Accounts(
                    name='Assets',
                    number=1000,
                    account_type='asset',
                    is_container=True,
                    is_root=True,
                    date_created=datetime.now(),
                    date_modified=datetime.now(),
                    description='Top-level asset account',
                    children={
                        Accounts(
                            name='Current',
                            number=1100,
                            account_type='asset',
                            is_container=True,
                            is_root=False,
                            date_created=datetime.now(),
                            date_modified=datetime.now(),
                            description='Container asset account for current assets',
                            children={
                                Accounts(
                                    name='Checking',
                                    number=1110,
                                    account_type='asset',
                                    is_container=False,
                                    is_root=False,
                                    date_created=datetime.now(),
                                    date_modified=datetime.now(),
                                    description='Checking account',
                                    children=set(),
                                    
                                )
                            }
                        ),
                        Accounts(
                            name='Investments',
                            number=1200,
                            account_type='asset',
                            is_container=True,
                            is_root=False,
                            date_created=datetime.now(),
                            date_modified=datetime.now(),
                            description='Container asset account for investments',
                            children={
                                Accounts(
                                    name='Development A',
                                    number=1210,
                                    account_type='asset',
                                    is_container=False,
                                    is_root=False,
                                    date_created=datetime.now(),
                                    date_modified=datetime.now(),
                                    description='Investment in Development A',
                                    children=set()
                                )
                            }
                        )
                    }                                        
                )
            ]
        )
    )

    

    s.add_all([company, ])
    s.commit()

    checking_acct_num = s.execute(text(f'SELECT id FROM accounts WHERE accounts.name="Checking"')).fetchone()[0]
    investment_acct_num = s.execute(text(f'SELECT id FROM accounts WHERE accounts.name="Development A"')).fetchone()[0]

    journal_entries = JournalEntries(
        description='ENTRY 1',
        memo='Memo for ENTRY 1',
        splits={
            Splits(
                description='Split 1 Debit',
                memo='Split 1 memo',
                account_key=investment_acct_num,
                debit=100.0,
                credit=0.0,
            ),
            Splits(
                description='Split 2 Credit',
                memo='Split 1 Credit memo',
                account_key=checking_acct_num,
                debit=0.0,
                credit=100.0,
            ),
        },
    )

    s.add_all([journal_entries])
    s.commit()

2025-05-21 14:53:54,684 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 14:53:54,687 INFO sqlalchemy.engine.Engine INSERT INTO company (company_name, company_type, date, description) VALUES (?, ?, ?, ?)
2025-05-21 14:53:54,687 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ('Test Company', 'LLC', '2025-05-21 14:53:54.675170', 'A test company')
2025-05-21 14:53:54,690 INFO sqlalchemy.engine.Engine INSERT INTO chart_of_accounts (company_key, name) VALUES (?, ?)
2025-05-21 14:53:54,690 INFO sqlalchemy.engine.Engine [generated in 0.00075s] (1, 'Test COA2')
2025-05-21 14:53:54,692 INFO sqlalchemy.engine.Engine INSERT INTO accounts (coa_key, parent_account_key, name, number, account_type, is_container, is_root, date_created, date_modified, description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-05-21 14:53:54,693 INFO sqlalchemy.engine.Engine [generated in 0.00066s] (1, None, 'Assets', 1000, 'asset', 1, 1, '2025-05-21 14:53:54.675175', '2025-05-21 14:53:54.675176', 'Top-leve

In [335]:
from sqlalchemy import text

engine.connect().execute(text('SELECT debit FROM splits WHERE splits.journal_entry_key=1')).fetchall()

2025-05-21 14:53:54,736 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 14:53:54,738 INFO sqlalchemy.engine.Engine SELECT debit FROM splits WHERE splits.journal_entry_key=1
2025-05-21 14:53:54,739 INFO sqlalchemy.engine.Engine [generated in 0.00260s] ()


[(4.0,), (4.0,)]

In [336]:
from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for t in inspector.get_table_names(schema=schema):
    print(engine.connect().execute(text(f'SELECT * from {t}')).fetchall())
    
for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

2025-05-21 14:53:54,749 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 14:53:54,751 INFO sqlalchemy.engine.Engine PRAGMA database_list
2025-05-21 14:53:54,752 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 14:53:54,754 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-21 14:53:54,755 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 14:53:54,755 INFO sqlalchemy.engine.Engine SELECT name FROM "main".sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-05-21 14:53:54,756 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-21 14:53:54,758 INFO sqlalchemy.engine.Engine ROLLBACK
2025-05-21 14:53:54,759 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-21 14:53:54,759 INFO sqlalchemy.engine.Engine SELECT * from accounts
2025-05-21 14:53:54,760 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ()
[(1, 1, None, 'Assets', 1000, 'asset', 1, 1, '2025-05-21 14:53:54.675175', '2025-05-21 14:53:54.675176', 'Top-level asset accoun